In [2]:
# IMPORTS
##########################

%load_ext autoreload
%autoreload 2

import agent
import environment
import doubledqn
import tools
import memory
import simulation
import multiprocessing
import pandas as pd
import os
import json

import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

import numpy as np
import matplotlib.pyplot as plt
import time
import itertools
from keras import optimizers 

def iter_params(**kwargs):
    keys = kwargs.keys()
    vals = kwargs.values()
    for instance in itertools.product(*vals):
        yield dict(zip(keys, instance))

Using TensorFlow backend.


In [ ]:
# ONE RUN
#############################################

param = { 
    
    "batch_size" : 30, # 30,50,60,[70]
    "target_update_freq" : 5000,#10000], # 10000,20000,30000,[40000]
    "gamma" : 0.99,# # 0.98,0.99,0.995,[0.999]
    "train_freq" : 1, # 2,3,4,[5]
    "max_size" : 100000,#,70000], # 20000,50000,70000,[100000]
    "max_ep_length" : 2000, # 1000,2000,3000,[4000]
    "policy" : "linDecEpsGreedy",#, "epsGreedy"],
    "eps" : 0.2,#, 0.3, 0.1],
    "delta_time" : 10,
    #"optimizer": [optimizers.RMSprop(lr= 0.001), optimizers.Adagrad(), optimizers.Adam()]
}

sumo_RL = simulation.simulator(
                    connection_label = "single_worker",
                    q_network_type = 'simple',
                    target_q_network_type = 'simple',
                    gamma = param["gamma"],
                    target_update_freq = param["target_update_freq"],
                    train_freq = param["train_freq"],
                    num_burn_in = 200,
                    batch_size = param["batch_size"],
                    optimizer = 'adam',
                    loss_func = "mse",
                    max_ep_length = param["max_ep_length"],
                    experiment_id = "check eval",
                    model_checkpoint = True,
                    opt_metric = None,

                   # environment parameters
                    network = "simple", # complex
                    net_file = "simple_cross.net.xml", # "complex_cross.net.xml",
                    route_file = "cross.rou.xml",
                    demand = "nominal",
                    state_shape = (1,15), #(1,11) or (1,29)
                    num_actions = 2, # 2 or 4
                    use_gui = False,
                    delta_time = param["delta_time"],

                   # memory parameters
                    max_size = param["max_size"],

                   # additional parameters

                    policy = param["policy"],
                    eps = param["eps"],
                    num_episodes = 1,
                    monitoring = True,
                    episode_recording = False,
                    hparams = param.keys())

sumo_RL.train()
# agent.load("./logs/First gridsearch/run_144/model_checkpoints/runFirst gridsearch_iter165000.h5")

# agent.ddqn.train(env = agent.env, num_episodes = 100, policy = agent.policy, connection_label = agent.connection_label)

#evaluation_results = agent.evaluate(runs=2, use_gui= False)
#evaluation_results
# data= agent.ddqn.evaluate(env = agent.env, policy = "greedy")

In [ ]:
sumo_RL.evaluate(runs=2)

In [ ]:
# GRIDSEARCH
###############################################################

experiment_id = "Complex_Balanced_Negative_Reward" 


param = { 
    
    "batch_size" : [30], #,50],#,50],# 30,50,60,[70]
    "target_update_freq" : [5000],#,40000],# 100000],  #10000], # 10000,20000,30000,[40000]
    "gamma" : [0.99],#,0.95],# # 0.98,0.99,0.995,[0.999]
    "train_freq" : [1],#,4], # 2,3,4,[5]
    "max_size" : [10000],#,70000], # 20000,50000,70000,[100000]
    "max_ep_length" : [1000],
    "policy" : ["linDecEpsGreedy","epsGreedy"],# "linDecEpsGreedy"],#, "epsGreedy"],
    "eps" : [0.3,0.1, 0.05],#, 0.3, 0.1],
    "delta_time" : [10,15],
    "reward" : ["balanced","negative"]
    #"optimizer": [optimizers.RMSprop(lr= 0.001), optimizers.Adagrad(), optimizers.Adam()]
}

param_grid = iter_params(**param)



def worker(input, output):
    """Runs through a chunk of the grid"""

    for position, args in iter(input.get, 'STOP'):
        result = worker_task(position, args)
        output.put(result)


def worker_task(position, args):
    """Tells the worker what to do with grid chunk"""
    # initialise all obje
    
    # print('Run', position + 1, '-- parameters', args)
    
    sumo_RL = simulation.simulator(
                    connection_label = position + 1,
                    q_network_type = 'simple',
                    target_q_network_type = 'simple',
                    gamma = args["gamma"],
                    target_update_freq = args["target_update_freq"],
                    train_freq = args["train_freq"],
                    num_burn_in = 1000,
                    batch_size = args["batch_size"],
                    optimizer = 'adam',
                    loss_func = "mse",
                    max_ep_length = args["max_ep_length"],
                    experiment_id = experiment_id,
                    model_checkpoint = True,
                    opt_metric = None,

                   # environment parameters
                    network = "complex", # complex
                    net_file = "complex_cross.net.xml", # "complex_cross.net.xml",
                    route_file = "cross.rou.xml",
                    demand = "rush",
                    state_shape = (1,41),#(1,41)
                    num_actions = 4, #4 
                    use_gui = False,
                    delta_time = args["delta_time"],
                    reward = args["reward"],

                   # memory parameters
                    max_size = args["max_size"],

                   # additional parameters

                    policy = args["policy"],
                    eps = args["eps"],
                    num_episodes = 200,
                    monitoring = True,
                    episode_recording = False,
                    hparams = args.keys())
    
    # print("training agent", position + 1)
    sumo_RL.train()
    # print("evaluating agent", position + 1)
    evaluation_results = sumo_RL.evaluate(runs = 5)

    return ({"run" : position + 1,
             "args" : args, 
             "eval_delay" : evaluation_results, 
             "eval_mean_delay" :evaluation_results["average_delay"]})


def gridsearch(param_grid):
    """Runs a parallelised gridsearch"""

    number_of_processes = multiprocessing.cpu_count()

    # Set up task list
    tasks = [(idx, val) for idx, val in enumerate(param_grid)]

    # Create queues
    task_queue = multiprocessing.Queue()
    done_queue = multiprocessing.Queue()

    # Submit tasks
    for task in tasks:
        task_queue.put(task)
    # Start worker processes
    for i in range(number_of_processes):
        print('Started process #', i + 1)
        multiprocessing.Process(target = worker,
                                args = (task_queue, done_queue)).start()
                    
    with open(os.path.join("./logs",experiment_id,"GS_results.json"), "a") as file:
            file.write('{ "results": [')
      
    # Get and print results
    results = []
    for i in range(len(tasks)):
        results.append(done_queue.get())
        
        with open(os.path.join("./logs",experiment_id,"GS_results.json"), "a") as file:
            json.dump(results[-1], file , indent=4) 
            if i != len(tasks)-1:
                file.write(",\n")
            
    with open(os.path.join("./logs",experiment_id,"GS_results.json"), "a") as file:
        file.write("]}")
                  
        #print('%s -- [RESULTS]: Run %s -- Parameters %s -- Mean duration %6.0f' % results[-1])
        
    # Tell child processes to stop
    for i in range(number_of_processes):
        task_queue.put('STOP')

    # Now combine the results
#     scores = [result[-1] for result in results]
#     lowest = min(scores)
#     winner = results[scores.index(lowest)]
#    return winner, results

multiprocessing.freeze_support()
gridsearch(param_grid)

Started process # 1
Started process # 2
Started process # 3
Started process # 4
Started process # 5
Started process # 6
Started process # 7
Started process # 8
Filling experience replay memory...
Filling experience replay memory...
Filling experience replay memory...
Filling experience replay memory...
Filling experience replay memory...
Filling experience replay memory...
Filling experience replay memory...
Filling experience replay memory...
...done filling replay memory
Run 2 -- running episode 1 / 200
...done filling replay memory
Run 1 -- running episode 1 / 200
...done filling replay memory
Run 4 -- running episode 1 / 200
...done filling replay memory
Run 3 -- running episode 1 / 200
...done filling replay memory
Run 8 -- running episode 1 / 200
...done filling replay memory
Run 7 -- running episode 1 / 200
...done filling replay memory
Run 6 -- running episode 1 / 200
...done filling replay memory
Run 5 -- running episode 1 / 200
Run 2 -- running episode 2 / 200
Run 1 -- runnin

Run 5 -- running episode 30 / 200
Run 7 -- running episode 32 / 200
Run 6 -- running episode 30 / 200
Run 2 -- running episode 27 / 200
Run 3 -- running episode 27 / 200
Run 7 -- running episode 33 / 200
Run 5 -- running episode 31 / 200
Run 8 -- running episode 30 / 200
Run 4 -- running episode 27 / 200
Run 1 -- running episode 28 / 200
Run 6 -- running episode 31 / 200
Run 2 -- running episode 28 / 200
Run 7 -- running episode 34 / 200
Run 3 -- running episode 28 / 200
Run 5 -- running episode 32 / 200
Run 8 -- running episode 31 / 200
Run 4 -- running episode 28 / 200
Run 6 -- running episode 32 / 200
Run 1 -- running episode 29 / 200
Run 2 -- running episode 29 / 200
Run 7 -- running episode 35 / 200
Run 5 -- running episode 33 / 200
Run 8 -- running episode 32 / 200
Run 3 -- running episode 29 / 200
Run 7 -- running episode 36 / 200
Run 4 -- running episode 29 / 200
Run 5 -- running episode 34 / 200
Run 6 -- running episode 33 / 200
Run 1 -- running episode 30 / 200
Run 2 -- runni

Run 3 -- running episode 55 / 200
Run 2 -- running episode 53 / 200
Run 4 -- running episode 51 / 200
Run 1 -- running episode 57 / 200
Run 7 -- running episode 69 / 200
Run 5 -- running episode 66 / 200
Run 8 -- running episode 60 / 200
Run 3 -- running episode 56 / 200
Run 6 -- running episode 61 / 200
Run 2 -- running episode 54 / 200
Run 7 -- running episode 70 / 200
Run 1 -- running episode 58 / 200
Run 5 -- running episode 67 / 200
Run 4 -- running episode 52 / 200
Run 8 -- running episode 61 / 200
Run 3 -- running episode 57 / 200
Run 6 -- running episode 62 / 200
Run 2 -- running episode 55 / 200
Run 7 -- running episode 71 / 200
Run 5 -- running episode 68 / 200
Run 1 -- running episode 59 / 200
Run 8 -- running episode 62 / 200
Run 4 -- running episode 53 / 200
Run 6 -- running episode 63 / 200
Run 3 -- running episode 58 / 200
Run 7 -- running episode 72 / 200
Run 5 -- running episode 69 / 200
Run 2 -- running episode 56 / 200
Run 1 -- running episode 60 / 200
Run 8 -- runni

Run 4 -- running episode 77 / 200
Run 2 -- running episode 81 / 200
Run 7 -- running episode 104 / 200
Run 1 -- running episode 86 / 200
Run 8 -- running episode 90 / 200
Run 5 -- running episode 100 / 200
Run 6 -- running episode 90 / 200
Run 3 -- running episode 85 / 200
Run 4 -- running episode 78 / 200
Run 7 -- running episode 105 / 200
Run 6 -- running episode 91 / 200
Run 5 -- running episode 101 / 200
Run 2 -- running episode 82 / 200
Run 8 -- running episode 91 / 200
Run 1 -- running episode 87 / 200
Run 3 -- running episode 86 / 200
Run 4 -- running episode 79 / 200
Run 7 -- running episode 106 / 200
Run 5 -- running episode 102 / 200
Run 6 -- running episode 92 / 200
Run 8 -- running episode 92 / 200
Run 2 -- running episode 83 / 200
Run 1 -- running episode 88 / 200
Run 3 -- running episode 87 / 200
Run 7 -- running episode 107 / 200
Run 5 -- running episode 103 / 200
Run 4 -- running episode 80 / 200
Run 8 -- running episode 93 / 200
Run 6 -- running episode 93 / 200
Run 2 

Run 7 -- running episode 137 / 200
Run 5 -- running episode 134 / 200
Run 6 -- running episode 120 / 200
Run 2 -- running episode 109 / 200
Run 1 -- running episode 113 / 200
Run 7 -- running episode 138 / 200
Run 8 -- running episode 120 / 200
Run 4 -- running episode 105 / 200
Run 3 -- running episode 112 / 200
Run 5 -- running episode 135 / 200
Run 6 -- running episode 121 / 200
Run 7 -- running episode 139 / 200
Run 8 -- running episode 121 / 200
Run 4 -- running episode 106 / 200
Run 2 -- running episode 110 / 200
Run 1 -- running episode 114 / 200
Run 5 -- running episode 136 / 200
Run 3 -- running episode 113 / 200
Run 6 -- running episode 122 / 200
Run 7 -- running episode 140 / 200
Run 8 -- running episode 122 / 200
Run 4 -- running episode 107 / 200
Run 5 -- running episode 137 / 200
Run 2 -- running episode 111 / 200
Run 1 -- running episode 115 / 200
Run 3 -- running episode 114 / 200
Run 6 -- running episode 123 / 200
Run 7 -- running episode 141 / 200
Run 8 -- running epi

...

Run 19 -- running episode 8 / 200
Evaluate 15 -- running episode 5 / 5
Run 21 -- running episode 3 / 200
Run 14 -- running episode 183 / 200
Run 20 -- running episode 5 / 200
Run 18 -- running episode 12 / 200
Run 16 -- running episode 187 / 200
Run 17 -- running episode 18 / 200
Run 19 -- running episode 9 / 200
Run 16 -- running episode 188 / 200
Filling experience replay memory...
Run 14 -- running episode 184 / 200
Run 20 -- running episode 6 / 200
Run 17 -- running episode 19 / 200
Run 21 -- running episode 4 / 200
Run 18 -- running episode 13 / 200
Run 14 -- running episode 185 / 200
Run 19 -- running episode 10 / 200
Run 17 -- running episode 20 / 200
Run 20 -- running episode 7 / 200
Run 16 -- running episode 189 / 200
Run 18 -- running episode 14 / 200
Run 21 -- running episode 5 / 200
Run 14 -- running episode 186 / 200
Run 17 -- running episode 21 / 200
Run 16 -- running episode 190 / 200
Run 19 -- running episode 11 / 200
Run 20 -- running episode 8 / 200
...done filling re

Run 20 -- running episode 33 / 200
Run 17 -- running episode 53 / 200
Run 23 -- running episode 9 / 200
Run 21 -- running episode 33 / 200
Run 19 -- running episode 40 / 200
Run 18 -- running episode 39 / 200
Run 17 -- running episode 54 / 200
Run 22 -- running episode 22 / 200
Run 21 -- running episode 34 / 200
Run 20 -- running episode 34 / 200
Run 24 -- running episode 5 / 200
Run 19 -- running episode 41 / 200
Run 23 -- running episode 10 / 200
Run 18 -- running episode 40 / 200
Run 17 -- running episode 55 / 200
Run 21 -- running episode 35 / 200
Run 20 -- running episode 35 / 200
Run 22 -- running episode 23 / 200
Run 19 -- running episode 42 / 200
Run 23 -- running episode 11 / 200
Run 17 -- running episode 56 / 200
Run 18 -- running episode 41 / 200
Run 21 -- running episode 36 / 200
Run 20 -- running episode 36 / 200
Run 24 -- running episode 6 / 200
Run 19 -- running episode 43 / 200
Run 17 -- running episode 57 / 200
Run 23 -- running episode 12 / 200
Run 21 -- running episo

Run 20 -- running episode 62 / 200
Run 18 -- running episode 67 / 200
Run 17 -- running episode 86 / 200
Run 24 -- running episode 29 / 200
Run 21 -- running episode 67 / 200
Run 19 -- running episode 72 / 200
Run 23 -- running episode 40 / 200
Run 22 -- running episode 46 / 200
Run 20 -- running episode 63 / 200
Run 17 -- running episode 87 / 200
Run 18 -- running episode 68 / 200
Run 21 -- running episode 68 / 200
Run 24 -- running episode 30 / 200
Run 19 -- running episode 73 / 200
Run 23 -- running episode 41 / 200
Run 17 -- running episode 88 / 200
Run 21 -- running episode 69 / 200
Run 20 -- running episode 64 / 200
Run 22 -- running episode 47 / 200
Run 18 -- running episode 69 / 200
Run 19 -- running episode 74 / 200
Run 23 -- running episode 42 / 200
Run 24 -- running episode 31 / 200
Run 17 -- running episode 89 / 200
Run 21 -- running episode 70 / 200
Run 20 -- running episode 65 / 200
Run 18 -- running episode 70 / 200
Run 22 -- running episode 48 / 200
Run 23 -- running ep

Run 17 -- running episode 118 / 200
Run 21 -- running episode 99 / 200
Run 24 -- running episode 55 / 200
Run 20 -- running episode 91 / 200
Run 22 -- running episode 72 / 200
Run 18 -- running episode 94 / 200
Run 19 -- running episode 103 / 200
Run 23 -- running episode 71 / 200
Run 17 -- running episode 119 / 200
Run 21 -- running episode 100 / 200
Run 24 -- running episode 56 / 200
Run 20 -- running episode 92 / 200
Run 22 -- running episode 73 / 200
Run 23 -- running episode 72 / 200
Run 17 -- running episode 120 / 200
Run 21 -- running episode 101 / 200
Run 19 -- running episode 104 / 200
Run 18 -- running episode 95 / 200
Run 24 -- running episode 57 / 200
Run 20 -- running episode 93 / 200
Run 17 -- running episode 121 / 200
Run 23 -- running episode 73 / 200
Run 22 -- running episode 74 / 200
Run 21 -- running episode 102 / 200
Run 19 -- running episode 105 / 200
Run 18 -- running episode 96 / 200
Run 24 -- running episode 58 / 200
Run 20 -- running episode 94 / 200
Run 17 -- 

Run 23 -- running episode 100 / 200
Run 18 -- running episode 121 / 200
Run 24 -- running episode 82 / 200
Run 22 -- running episode 99 / 200
Run 21 -- running episode 130 / 200
Run 19 -- running episode 133 / 200
Run 17 -- running episode 149 / 200
Run 20 -- running episode 120 / 200
Run 23 -- running episode 101 / 200
Run 18 -- running episode 122 / 200
Run 24 -- running episode 83 / 200
Run 22 -- running episode 100 / 200
Run 21 -- running episode 131 / 200
Run 19 -- running episode 134 / 200
Run 20 -- running episode 121 / 200
Run 17 -- running episode 150 / 200
Run 23 -- running episode 102 / 200
Run 18 -- running episode 123 / 200
Run 24 -- running episode 84 / 200
Run 22 -- running episode 101 / 200
Run 21 -- running episode 132 / 200
Run 19 -- running episode 135 / 200
Run 17 -- running episode 151 / 200
Run 20 -- running episode 122 / 200
Run 23 -- running episode 103 / 200
Run 18 -- running episode 124 / 200
Run 24 -- running episode 85 / 200
Run 21 -- running episode 133 / 2

In [ ]:
curr_eps = 
total_it = 100
for itr in range(100)
    if itr < total_it:
        curr_eps = (curr_eps - init_eps) / total_it * itr + init_eps
    else :
        curr_eps = init_eps

In [ ]:
decay_test = []
eps_test = 0.8
for i in range(300000):
    eps_test *= omega_test ** i
    decay_test.append(eps_test)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(np.arange(300000),decay_test)

In [ ]:
first_results = pd.DataFrame(results).sort_values(by = 3)

In [ ]:
# Get max evaluate results

res = {}
with open('./logs/Test_evaluate/GS_results.json') as file:
    data = json.load(file)
    for run in data['results']:
        run_no = run["run"]
        res[f"{run_no}"] = run["eval_mean_delay"]
        
res

In [ ]:
pd.read_json('./logs/Test_evaluate/GS_results.json',)